In [1]:
import os
import ctools
import numpy as np
import random
import math
from astropy.io import fits
from lxml import etree
if not os.path.isdir('../data'):
    os.mkdir('../data')
if not os.path.isdir('../models/test'):
    os.mkdir('../models/test')

In [2]:
def run_ctobssim(model, output,pointing = (83.6331, 22.5145), energy=(0.03, 5), time=(0, 1200), fov=5, caldb='prod3b-v2', irf='South_z40_0.5h', seed=1):
    sim = ctools.ctobssim()
    sim["inmodel"] = model #inserisce un modello pre-esistente all'interno del tool di generazione mappe
    sim["outevents"] = output #estrae una mappa creata in estensione .fits
    sim["caldb"] = caldb #calibration database, inserito nella fase di configurazione come zip
    sim["irf"] = irf #Instrumental response function, mappa tra flusso di fotoni in ingresso e gli eventi detected
    sim["ra"] = pointing[0] #Right Ascension of CTA pointing (dove punta nello spazio il telescopio)
    sim["dec"] = pointing[1] #Declination of CTA pointing (dove punta nello spazio il telescopio)
    sim["rad"] = fov #field of view, simulation cone radius, 5 rad mi pare un botto
    sim["tmin"] = time[0] #start time
    sim["tmax"] = time[1] #stop time of the exposure
    sim["emin"] = energy[0] #Lower energy limit of simulated events (in TeV).
    sim["emax"] = energy[1] #Upper energy limit of simulated events (in TeV).
    sim["seed"] = seed #random number generator for the simulations with a seed value provided by the hidden seed parameter
    sim.execute()
    # if you want to run in memory:
    #sim.run()
    
def run_skymap(obs, output, energy=(0.03, 5), roi=5, caldb='prod3b-v2', irf='North_z20_0.5h', wbin=0.02):
    nbin = int(roi*2/wbin)
    skymap = ctools.ctskymap()
    skymap['inobs'] = obs
    skymap['outmap'] = output
    skymap['irf'] = irf 
    skymap['caldb'] = caldb #calibration database, inserito nella fase di configurazione come zip
    skymap['emin'] = energy[0]
    skymap['emax'] = energy[1]
    skymap['usepnt'] = True #use pointing
    skymap['nxpix'] = nbin #Size of the Right Ascension / Galactic longitude axis (in pixels).
    skymap['nypix'] = nbin #Size of the Declination / Galactic latitude axis (in pixels).
    skymap['binsz'] = wbin #Pixel size (in degrees/pixel).
    skymap['bkgsubtract'] = 'IRF' #da testare 'NONE' o 'IRF'
    skymap.execute()
    # if you want to run in memory:
    #sim.run()
    
def truncate(num, n):
    integer = int(num * (10**n))/(10**n)
    return float(integer)

In [3]:
def modify_source_position(ra, dec, index):
    tree = etree.parse('../models/group3.xml')
    root = tree.getroot()
    Ra = root[0][1][0]
    Ra.set("value", str(ra))
    Dec = root[0][1][1]
    Dec.set("value", str(dec))
    et = etree.ElementTree(root)
    name = '../models/test/group3_' + str(index) + '.xml' 
    et.write(name, pretty_print=True)
        
def modify_two_source_position(ra, dec, index, ra2, dec2):
    #find the second point in the xml file 
    tree = etree.parse('../models/crab2sources.xml')
    root = tree.getroot()
    Ra = root[0][1][0]
    Ra.set("value", str(ra))
    Dec = root[0][1][1]
    Dec.set("value", str(dec))
    #modify the value and write the model
    Ra2 = root[1][1][0]
    Ra2.set("value", str(ra2))
    Dec2 = root[1][1][1]
    Dec2.set("value", str(dec2))
    et = etree.ElementTree(root)
    name = '../models/test/crab2sources_' + str(index) + '.xml' 
    et.write(name, pretty_print=True)
        

In [4]:
def create_new_model(i, pointing, maxAng=4.5, twoPoints="false"):
    
    #random with Module and Angle 
    angle = random.random() *2 *math.pi
    cos = math.cos(angle)
    sin = math.sin(angle)
    mod = random.random()*maxAng

    ra = pointing[0] + truncate(sin*mod,4)
    dec = pointing[1] + truncate(cos*mod,4)
    
    if(twoPoints == "true"):
        ipo = 0
        while(ipo<2): #to modify ------------------------------------------
            angle = random.random() *2 *math.pi
            cos = math.cos(angle)
            sin = math.sin(angle)
            mod = random.random()*maxAng
            
            ra2 = pointing[0] + truncate(sin*mod,4)
            dec2 = pointing[1] + truncate(cos*mod,4)
            ipo = ((ra-ra2)**2 + (dec-dec2)**2)**1/2
        modify_two_source_position(ra,dec,i,ra2,dec2)
        
    #richiamiamo la modifica e creazione di nuovi modelli coi parametri genrati pocanzi
    modify_source_position(ra,dec,i)

#tree = etree.parse('../models/crab4sigmas_new.xml')
#root = tree.getroot()
#print(etree.tostring(tree))

## Source simulation and map

Before this step you should have modified the source model (i.e., crab.xml) with the correct prefactor. You can do so manually or use the provided class in <code>sagsci</code> package for which you can find an example in <code>4_update_source_model.ipynb</code>. For a shortcut I have provided in <code>models</code> folder a template source model with the correct Prefactor for a 4 sigmas simulation of the Crab Nebula. The suggestion is to never go below this value.

In [10]:
count=1
pointing = (83.6331, 22.5145)
for i in range(count):
    create_new_model(i,pointing)
    model = '../models/test/group3_' + str(i) + '.xml'
    run_ctobssim(model=model, output='../data/group3_sim.fits')
    obs = '../data/group3_sim.fits'
    run_skymap(obs=obs, output='../data/group3_sky_' + str(i) + '.fits')
os.remove(obs)

## Background simulation and map

We can run the simulation of the background and then use the observation produced to run create the skymap. The simulation will be used in tutorial <code>3_find_prefactor.ipynb</code> to evaluate the background level and find the prefactor required to simulate a source with given significance. Alternatively you can used the template model <code>crab4sigmas.xml</code> in <code>models</code>. The skymap can be used as visual inspection of the simulation and as part of the training of the ML algorithm.

In [9]:
count=10
pointing = (83.6331, 22.5145)
for i in range(count):
    model = '../models/bkg_irf.xml'
    run_ctobssim(model=model, output='../data/bkg_test_sim.fits',seed= int(random.random()*100))
    obs = '../data/bkg_test_sim.fits'
    run_skymap(obs=obs, output='../data/bkg_test_sky_' + str(i) + '.fits')
os.remove(obs)

In [7]:
count=10
pointing = (83.6331, 22.5145)
for i in range(count):
    create_new_model(i,pointing,twoPoints="true")
    model = '../models/test/crab2sources_' + str(i) + '.xml'
    run_ctobssim(model=model, output='../data/crab2sources_sim.fits')
    obs = '../data/crab2sources_sim.fits'
    run_skymap(obs=obs, output='../data/crab2sources_sky_' + str(i) + '.fits')
os.remove(obs)